In [26]:
import numpy as np
from typing import Dict, List, Tuple, Any, Callable
import sys, os, random
# Add the project root directory to Python path
# Set the project root directory to the parent directory of the current working directory
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

from utils.data import get_fixed_data
from utils.WindProcess import wind_model
from utils.PriceProcess import price_model

def get_fixed_data():
    """
    Returns the fixed data for the energy hub simulation.
    """
    num_timeslots = 24
    return {
        # Conversion efficiencies
        'conversion_p2h': 0.9,
        'conversion_h2p': 0.8,

        # Hydrogen storage capacity
        'hydrogen_capacity': 15,
        
        'p2h_max_rate': 5,
        'h2p_max_rate': 5,

        # Electrolyzer cost
        'electrolyzer_cost': 1, 

        # Wind model parameters
        'target_mean_wind': 4.5,
        'wind_reversion_strength': 0.15,
        'extreme_event_prob_wind': 0.03,

        # Price model parameters
        'mean_price': 35,
        'price_reversion_strength': 0.12,
        'wind_influence_on_price': -0.6,
        'price_cap': 90,  
        'price_floor': 0,
        
        
        'num_timeslots': num_timeslots,
        'demand_schedule': [5 + 2 * np.sin(2 * np.pi * t / 24) for t in range(num_timeslots)]

        
    }

data = get_fixed_data()

In [32]:

def sample_representative_state_pairs(I, T, data):
  
    trajectories = []

    for _ in range(I):
        trajectory = []

        # Initialize wind and price values
        current_wind = np.random.uniform(0, 10)
        previous_wind = current_wind + np.random.normal()
        current_price = np.random.uniform(10, 70)
        previous_price = current_price + np.random.normal()

        for t in range(T):
            wind_t = round(wind_model(current_wind, previous_wind, data), 2)
            price_t = round(price_model(current_price, previous_price, wind_t, data), 2)

            # Sample endogenous state
            storage = round(np.random.uniform(0, data['hydrogen_capacity']), 2)
            status = np.random.choice([0, 1])

            z_t = (wind_t, price_t)
            y_t = (storage, status)
            trajectory.append((z_t, y_t))

            # Update wind and price
            previous_wind = current_wind
            current_wind = wind_t
            previous_price = current_price
            current_price = price_t

        trajectories.append(trajectory)

    return trajectories

print(sample_representative_state_pairs(1, 1, data))

[[((5.04, 48.13), (13.29, np.int64(1)))]]


In [ ]:
def create_experiments(
        num_experiments: int, 
        data: Dict[str, Any]
    ) -> Tuple[List[int], np.ndarray, np.ndarray]:
    
    trajectories = []

    wind_trajectories = np.zeros((num_experiments, data['num_timeslots']))
    price_trajectories = np.zeros((num_experiments, data['num_timeslots']))
    
    for e in range(num_experiments):
        # Initialize first two values for each trajectory
        trajectory = []

        wind_trajectories[e, 0] = 4
        wind_trajectories[e, 1] = 5
        price_trajectories[e, 0] = 28
        price_trajectories[e, 1] = 30
        
        # Generate trajectories for the experiment
        for t in range(2, data['num_timeslots']):
            wind_trajectories[e, t] = wind_model(
                wind_trajectories[e, t-1],
                wind_trajectories[e, t-2],
                data
            )
            price_trajectories[e, t] = price_model(
                price_trajectories[e, t-1],
                price_trajectories[e, t-2],
                wind_trajectories[e, t],
                data
            )
        # Sample endogenous state
            h_storage = np.random.uniform(0, data['hydrogen_capacity'])
            elzr_status = np.random.choice([0, 1])

            z_t = (wind_trajectories[e, t], price_trajectories[e, t])
            y_t = (h_storage, elzr_status)
            trajectory.append((z_t, y_t))

        trajectories.append(trajectory)
    # Convert trajectories to numpy arrays
    wind_trajectories = np.array(wind_trajectories)
    price_trajectories = np.array(price_trajectories)
    # Ensure the trajectories have the correct shape
    assert wind_trajectories.shape == (num_experiments, num_timeslots)
    assert price_trajectories.shape == (num_experiments, num_timeslots)
    # Ensure the trajectories are not empty
    assert wind_trajectories.size > 0
    assert price_trajectories.size > 0
    # Ensure the trajectories are not NaN
    assert not np.isnan(wind_trajectories).any()
    assert not np.isnan(price_trajectories).any()
    # Ensure the trajectories are not infinite
    assert not np.isinf(wind_trajectories).any()
    assert not np.isinf(price_trajectories).any()
    
    return trajectories

print(create_experiments(1, data))

[[((np.float64(4.702752025352409), np.float64(32.78796564284371)), (11.73774401815028, np.int64(0))), ((np.float64(4.711493112143567), np.float64(28.775567483876234)), (1.6638126364380228, np.int64(0))), ((np.float64(5.834158112146561), np.float64(21.25694722297153)), (0.7809565475588087, np.int64(1))), ((np.float64(6.2070880686992975), np.float64(14.407823799657164)), (5.941660285370273, np.int64(0))), ((np.float64(8.629682645931283), np.float64(9.18332738608546)), (4.766704910167131, np.int64(1))), ((np.float64(9.256001605350676), np.float64(1.9277116633032947)), (8.913614067139056, np.int64(0))), ((np.float64(11.242440533273374), np.float64(4.027972195926711)), (7.107114305887409, np.int64(1))), ((np.float64(11.122626492462626), np.float64(2.6002984895705494)), (11.405886799037786, np.int64(1))), ((np.float64(10.186187830753223), np.float64(2.037100903660023)), (5.559917646075419, np.int64(0))), ((np.float64(8.116191338765844), np.float64(0.25672727392656775)), (13.375512501329489, 